# Description
For each WormBase id find the genomic_position
and write the 2,000 nucleotides before the genomic start position

Note: that the start may be directionally dependent


In [ ]:
%%bash
# Download Gene IDs from Wormbase
wormbase_version="WS293"
output_dir="./output"
base_url="https://downloads.wormbase.org/releases/${wormbase_version}/species/c_elegans/PRJNA13758"
gene_ids="c_elegans.PRJNA13758.${wormbase_version}.geneIDs.txt.gz"
wget -nv -P "${output_dir}" "${base_url}/annotation/${gene_ids}"
gunzip --force ${output_dir}/${gene_ids}


In [ ]:
%%bash
# Create GeneIDs.csv from the downloaded txt file
wormbase_version="WS293"
output_dir="./output"
gene_ids="c_elegans.PRJNA13758.${wormbase_version}.geneIDs"

gene_ids_txt="${output_dir}/${gene_ids}.txt"
gene_ids_csv="${output_dir}/${gene_ids}.csv"

awk -F',' '$5=="Live" {print $2","$3","$4","$6}' "$gene_ids_txt" > "$gene_ids_csv"
awk 'BEGIN {print "Wormbase_Id,Gene_name,Sequence_id,Gene_Type"} 1' "${gene_ids_csv}" > temp && mv temp "${gene_ids_csv}"


# Genomic Position Data Retrieval Script
-----------


### Overview

Retrieves genomic position data for C. elegans genes from Wormbase API.


### Functionality

This script:

* Reads a CSV file containing Wormbase gene IDs
* Queries the Wormbase API for genomic position data (chromosome, start position, stop position) for each gene
* Writes the results to a new CSV file


### Requirements

* Wormbase API access
* Input CSV file containing Wormbase gene IDs: `./output/c_elegans.PRJNA13758.WS293.geneIDs.csv`
* Output CSV file for genomic position data: `./output/genomic_position.csv`




In [ ]:
import os
import sys
import json
import pandas as pd

# Add pub_worm directory to the Python path
sys.path.append("/Users/dan/Code/Python/pub_worm")
from pub_worm.wormbase.wormbase_api import WormbaseAPI
from pub_worm.ensembl.ensembl_api import get_sequence_region, create_fasta

wormbase_version="WS293"
gene_ids_csv = f"./output/c_elegans.PRJNA13758.{wormbase_version}.geneIDs.csv"


wormbase_api = WormbaseAPI("field", "gene", "genomic_position")
gene_ids_df = pd.read_csv(gene_ids_csv)

results = []
total_ids = len(gene_ids_df['Wormbase_Id'])

for idx, wormbase_id in enumerate(gene_ids_df['Wormbase_Id']):
    wormbase_data_result = wormbase_api.get_wormbase_data(wormbase_id)
    if "genomic_position" in wormbase_data_result:
        chromosome = wormbase_data_result["genomic_position"]["seqname"]
        start_pos = wormbase_data_result["genomic_position"]["start"]
        stop_pos = wormbase_data_result["genomic_position"]["stop"]
    else:
        print(f"NO genomic_position for {wormbase_id}")
    results.append((wormbase_id,chromosome,start_pos,stop_pos))
    
    # Print a progress update every 50 iterations
    if (idx + 1) % 50 == 0 or (idx + 1) == total_ids:  # Also print for the final iteration
        print(f"Processed {idx + 1} Wormbase IDs of {total_ids}")

    
df_results = pd.DataFrame(results, columns=['Wormbase_Id', 'Chromosome', 'Start_Pos', 'Stop_Pos'])

# Write the DataFrame to a CSV file
df_results.to_csv('./output/genomic_position.csv', index=False)


# Retrieves genomic sequences for promoter regions
---------------

### Overview

Retrieves genomic sequences for promoter regions of given WormBase IDs using the Ensembl API.

### Functionality

This script:

* Reads a CSV file containing WormBase IDs, chromosome numbers, and genomic positions.
* Adjusts positions to extract 2000 bp promoter regions (upstream or downstream).
* Uses asyncio to fetch sequences from Ensembl API in parallel.
* Divides the input DataFrame into chunks and processes them concurrently using a ProcessPoolExecutor.
* Combines results and saves them to a new CSV file containing WormBase IDs and corresponding genomic sequences.

### Additional Requirements

* This code does not run in the Notebook do to limitation with Jupyter
* Use not_a_notebook.py as Jupyter can not run this code!!
* python -u "/Users/dan/Code/Python/pub_worm/notebooks/not_a_notebook.py"



# Convert PFM file to a PWM Dictionary
---------------

### Overview

Convert Position Frequency Matrix (PFM) file to a Position Weight Matrix (PWM) Dictionary

### Functionality

This script:

* Reads a CSV file containing PFM downloaded from https://jaspar.elixir.no/.
* Use BioPython to convert PFM to PWD.
* Create a Dictionary for down stream use

In [ ]:
from Bio import motifs
from Bio.Seq import Seq
import io

def read_pwm_file(file_path):
    pwm_dict = {}
    with open(file_path, 'r') as file:
        pwm_key = None
        pfm_data = ""

        for line in file:
            # Check if the line starts with '>'
            if line.startswith('>'):
                # If we already have a key and a matrix, process the previous one
                if pwm_key is not None and pfm_data:
                    pwm_dict[pwm_key] = convert_pfm_to_pwm(pfm_data)
                # Start a new PWM entry
                pwm_key = line.strip()[1:]  # Remove the '>' character
                pfm_data = ""  # Reset matrix for the new PWM
            else:
                pfm_data += line

        # Don't forget to process the last PWM matrix
        if pwm_key is not None and pfm_data:
            pwm_dict[pwm_key] = convert_pfm_to_pwm(pfm_data)

    return pwm_dict

def convert_pfm_to_pwm(pfm_data):
    pfm_file = io.StringIO(pfm_data)
    pwm = motifs.read(pfm_file, "pfm")
    pwm = pwm.counts.normalize(pseudocounts=0.1)  # Add pseudocounts to avoid division by zero
    pwm = pwm.log_odds()
    return pwm

# Example usage
file_path = './output/JASPAR2024_combined_matrices_1248851_pfm.txt'
file_path = "./output/MA2156.1.xbp-1.pfm"

pwm_dict = read_pwm_file(file_path)

# Print the PWM for a particular motif
print(f"{len(pwm_dict)}")

for key, pwm in pwm_dict.items():
    print(f'Motif: {key}\nPWM:\n{pwm}\n')

In [ ]:

# Scan the sequence for motif matches

def search_promoter_sequence(pwm_dict, promoter_sequence_str):
    promoter_sequence = Seq(promoter_sequence_str)
    found_keys={}
    for key, pwm in pwm_dict.items():
        matches = pwm.search(promoter_sequence)
        for match in matches:
            if key in found_keys:
                found_keys[key].append(matches)
            else:
                found_keys[key] = [matches]
    return found_keys
 


promoter_sequence_str = "AAGCATCACCGGAAGAACAACGAACTGTAAGTGAGATTATTTTAACAAATTTCCCGGCTTAAAACGTTTACAATTCTAACTTCTTTTTTTAGATAATGGAAGGCCTTGGTCATGTACATCGTCTTCAATGCACTCACCTAATTGAGAATAGCACACTGATTCCTAAACTACAAGAAATTAAATTTGACTTTGCAATTCATGAAGTATTCGATTCTTGTGGAGTCGGTATTCTTGAGGTAATCGGTGTACAGAAGACTGTTATTGTATCATCAACTGGACCAATGGATGTTGTCCCAATTACACTTGGAATATCTGATACATTGAACACTCCATGTTAGTTTGACACAGAATGGAAATGATTTACTATGAAACCAATTTCAGCTCTATTATCGGATTATGGAAGTTACCTATCATTTTTTGAAAAACGAAGAAATCTCAAGTTTCTATCTGGAATGCTCAATTTCCACGAAATGCAGGACTCAATGATATCTCCGCTTTTCAAAAAATATTATGGATTGAAAAAACCTACTGGTGAAATAATGAGACAAGCAAATTTATTGTTTTATAATATTCATGAAGGATCAGATGGAATGAGAATGAGAGGACGAAGAAGTTTTGATATTGGAGGGATTGCTTTTAAGGATCAAAAGAATTTGACTATGGTATTTTGCCTTTTTGCAAATTTTTTGAATAGCGCCTGAAGCAGTTTGCAAGCTTTACAATAAGATTATGGTGGTATTGATTTTTAATTCAGGAGTATCAAACTCTTCTCTCTGATCCTCGTCCTAAAGTGCTCGTATCATTTGGTACTGCAGCCACATCATCTCATATGCCTCAAAATTTGAAAAATTCGTTGATGACTGCAATGAAACAAATGAACAATGTTTTGTTCATATGGAAATATGAAATGGAAGATAATTTCACAAAACAGGAGGAGCTCACAACAAATATAATTTTCAAAAAGTTTTTGCCGCAGACTGATTTATTAGGTGAGTGATTT"
found_keys = search_promoter_sequence(pwm_dict, promoter_sequence_str)  
print(len(found_keys))
for found_key in found_keys:   
    print(f"Motif {found_key} found matches {len(found_keys[found_key])}")
    
    



In [ ]:
import pandas as pd
from Bio import motifs
from Bio.Seq import Seq
import io

genomic_sequences_1000_df = pd.read_csv("./output/genomic_sequences_1000.csv")
pwm_dict = read_pwm_file("./output/MA2156.1.xbp-1.pfm")

top_genes=["WBGene00002016", "WBGene00002017", "WBGene00002019", "WBGene00022382", "WBGene00002021", "WBGene00002015", "WBGene00000214", "WBGene00010470", 
           "WBGene00012683", "WBGene00000215", "WBGene00002018", "WBGene00009430", "WBGene00000217", "WBGene00006533", "WBGene00006959", "WBGene00015756", 
           "WBGene00008850", "WBGene00021979", "WBGene00013035", "WBGene00015913", "WBGene00001091", "WBGene00003903", "WBGene00004438", "WBGene00019068"]


for top_gene in top_genes: 
    top_gene_row = genomic_sequences_1000_df[genomic_sequences_1000_df['Wormbase_Id'] == top_gene]
    #sequence_str = ' '.join(top_gene['Sequence'].astype(str))
    sequence_str = top_gene_row['Sequence'].iloc[0]
    wormbase_id_str = top_gene_row['Wormbase_Id'].iloc[0]

    found_keys = search_promoter_sequence(pwm_dict, sequence_str)  

    for found_key in found_keys:   
        print(f"{wormbase_id_str} found matches {len(found_keys[found_key])}")



In [ ]:
import pandas as pd

wormbase_version="WS293"
gene_ids_csv = f"./output/c_elegans.PRJNA13758.{wormbase_version}.geneIDs.csv"
gene_ids_df = pd.read_csv(gene_ids_csv)
top_30_down_df = pd.read_csv("./output/top_30_down.csv", header=None, names=['Wormbase_Id'])
merged_df = top_30_down_df.merge(gene_ids_df, how='left', left_on='Wormbase_Id', right_on='Wormbase_Id')
merged_df[['Gene_name','Wormbase_Id']].to_csv('./output/top_30_down_seq.csv', sep='\t', index=False)